In [ ]:
!pip install pandas numpy scikit-learn
!pip install konlpy
!pip install torch             # 또는 GPU:
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu117
!pip install sentence-transformers
!pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlin

In [ ]:
# -*- coding: utf-8 -*-
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # BLAS 스레드 제한

import re
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# 1) 데이터 로드 및 컬럼 정리
df = pd.read_csv('your_reviews.csv', usecols=['리뷰내용'])
df = df.rename(columns={'리뷰내용':'review'})
df['review'] = df['review'].fillna('').astype(str)

# 2) 불용어 사전
STOPWORDS = set(['이','가','은','는','도','에','와','과','로','으로','의','를','을','하다'])

# 3) 형태소 분석기 (Okt)
okt = Okt()

def preprocess(text: str) -> str:
    # 특수문자, 이모지, 숫자 제거
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

def tokenize(text: str) -> list[str]:
    # 형태소 분석 + 원형(stem) + 명사/형용사/동사만 + 불용어 제거
    pos = okt.pos(text, stem=True)
    return [
        w for w,t in pos
        if t in ('Noun','Adjective','Verb') and w not in STOPWORDS
    ]

# 4) 키워드 추출용 SBERT 모델
KW_MODEL = SentenceTransformer('jhgan/ko-sroberta-multitask')

def extract_keywords(doc: str, top_k: int = 5) -> list[str]:
    clean = preprocess(doc)
    toks  = tokenize(clean)
    if not toks:
        return []
    # 4-gram 후보가 아니라 1-gram만
    tfidf = TfidfVectorizer(ngram_range=(1,1)).fit([' '.join(toks)])
    candidates = tfidf.get_feature_names_out()

    # 리뷰 & 후보 임베딩
    emb_doc = KW_MODEL.encode(doc, convert_to_tensor=True)
    emb_kw  = KW_MODEL.encode(candidates, convert_to_tensor=True)

    # 코사인 유사도 → NumPy
    scores = util.pytorch_cos_sim(emb_doc, emb_kw)[0]
    scores_np = scores.cpu().detach().numpy()

    # 상위 top_k 인덱스
    idx = np.argpartition(-scores_np, top_k)[:top_k]
    return [candidates[i] for i in idx]

# 5) 감성분석 파이프라인 (긍/부정)
sentiment = pipeline(
    'sentiment-analysis',
    model='monologg/koelectra-base-v3-discriminator',
    tokenizer='monologg/koelectra-base-v3-discriminator',
    device=-1  # CPU 모드
)

# 6) DataFrame에 적용
df['cleaned']   = df['review'].map(preprocess)
df['tokens']    = df['cleaned'].map(tokenize)
df['keywords']  = df['review'].map(lambda x: extract_keywords(x, top_k=5))
df['sentiment'] = df['review'].map(lambda x: sentiment(x)[0]['label'])

# 7) 결과 확인
print(df[['review','keywords','sentiment']].head(10))

# 8) CSV 저장
df.to_csv('reviews_with_keywords_sentiment.csv', index=False)


In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/V1_Recommend System/관광지 리뷰 크롤링/관광지_리뷰_크롤링_ALL.csv')

data

In [ ]:
# -*- coding: utf-8 -*-
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # BLAS 스레드 제한

import re
import pandas as pd
import numpy as np
from glob import glob
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# 1) 데이터 로드 및 컬럼 정리
file_paths = glob('/content/drive/MyDrive/V1_Recommend System/관광지 리뷰 크롤링/관광지_리뷰_크롤링_ALL.csv')

df = pd.concat([
    pd.read_csv(fp, encoding='utf-8', engine='python')
    for fp in file_paths
], ignore_index=True)

# 2) 리뷰 컬럼이름 통일 & 빈 값 제거
if '리뷰내용' not in df.columns:
    raise ValueError("컬럼 '리뷰내용'이 없습니다.")
df = df[['리뷰내용']].dropna(subset=['리뷰내용'])
df = df.rename(columns={'리뷰내용':'review'})
df['review'] = df['review'].astype(str)
# 2) 불용어 사전
STOPWORDS = set(['이','가','은','는','도','에','와','과','로','으로','의','를','을','하다'])

# 3) 형태소 분석기
okt = Okt()
def preprocess(text: str) -> str:
    text = re.sub(r'[^\w\s]', ' ', text)  # 특수문자 제거
    text = re.sub(r'\d+', ' ', text)      # 숫자 제거
    return re.sub(r'\s+', ' ', text).strip()

def tokenize(text: str) -> list[str]:
    pos = okt.pos(text, stem=True)
    return [
        word for word, tag in pos
        if tag in ['Noun','Adjective','Verb'] and word not in STOPWORDS
    ]

# 4) 키워드 추출 함수 (SBERT)
KW_MODEL = SentenceTransformer('jhgan/ko-sroberta-multitask')
def extract_keywords(doc: str, top_k: int = 5) -> list[str]:
    # 전처리 & 토크나이즈
    clean = preprocess(doc)
    toks  = tokenize(clean)
    if not toks:
        return []

    # TF-IDF 로 후보 만들기 (에러 방어)
    try:
        tfidf = TfidfVectorizer(ngram_range=(1,1)).fit([' '.join(toks)])
        candidates = tfidf.get_feature_names_out()
    except ValueError:
        return []

    if len(candidates) == 0:
        return []

    # 실제 뽑을 키워드 개수
    k = min(top_k, len(candidates))

    # 문장과 후보 임베딩
    emb_doc = KW_MODEL.encode(doc, convert_to_tensor=True)
    emb_kw  = KW_MODEL.encode(candidates, convert_to_tensor=True)

    # 코사인 유사도 상위 k개
    scores = util.pytorch_cos_sim(emb_doc, emb_kw)[0]
    topk_idx = scores.topk(k).indices.cpu().numpy()
    return [candidates[i] for i in topk_idx]

# 5) 감성분석 파이프라인
sentiment = pipeline(
    'sentiment-analysis',
    model='monologg/koelectra-base-v3-discriminator',
    tokenizer='monologg/koelectra-base-v3-discriminator',
    device=-1
)

# 6) DataFrame에 적용
df['cleaned']   = df['review'].map(preprocess)
df['tokens']    = df['cleaned'].map(tokenize)
df['keywords']  = df['review'].map(lambda x: extract_keywords(x, top_k=5))
df['sentiment'] = df['review'].map(lambda x: sentiment(x)[0]['label'])

# 7) 결과 확인
print(df[['review','keywords','sentiment']].head())

# 8) CSV로 저장
df.to_csv('/content/drive/MyDrive/TL_CSV/음식점_all_reviews_with_keywords_sentiment_all.csv', index=False)


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/TL_CSV/음식점_all_reviews_with_keywords_sentiment_all.csv')
df

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/TL_CSV/음식점_all_reviews_with_keywords_sentiment_all.csv')

data

## Review Data :
- 작성자 id
- 장소 id
- 작성자 이름
- 작성 시간
- 별점
- 리뷰 내용
- 작성자가 올린 사진

-> 최소 1개의 데이터


## Meta Data :
- 장소 id
- 장소명
- 주소
- 장소 설명글
- 위도
- 경도
- 카테고리
- 평균 별점
- 총 리뷰수
- 영업시간
- url

-> 평점 4.0 이상의 데이터들이 80% 차지


## 추천 시스템
1. Item Pool
2. Generate Candidates
3. Intention Predict
4. Ranking

# Item 기반 추천 시스템(분류기 모델)
- 아이템간의 연관성을 찾기 위해 연관 규칙 분석 사용
- Review Table 예시
-  장소 id / user_id / 평점 / review / 분기(작성 시간)

-> Co - review 데이터 생성
- 동일 연도 - 분기를 기준으로 서로 다른 장소 두 곳(장소 id1, 장소 id2) 공통으로 리뷰를 작성한 유저 수 카운트

- 분류기 모델 feature
1. 연도
2. 분기
3. 두 장소의 거리
4. 장소_id 1/2의 직전 분기 리뷰 수
5. 장소_id 1/2의 직전 분기 유니크한 user 수
6. 직전 분기의 co_review 수

-> GBDT 모델 사용


- 성능 지표
1. MAP@k5 : 추천된 상위 5개 장소 중 사용자가 실제로 방문한 장소가 나타날 확률
2. NDCG@5 : 사용자가 실제로 방문한 장소가 추천 리스트 상위에 얼마나 자주 나타나는지
3. Precision@5 : 상위 5개 추천 장소 중 사용자가 실제로 방문한 장소의 비율
4. Rdcall@5 : 전체 방문한 장소 중 상위 5개 추천에 포함된 장소의 비율

- RAG 프로세스
Unstructured Data (Review / Meta) -> Embedding model(Open AI) -> Meta(파시스 메타 DB)

- Vector DB
-> FAISS
- 대용량 고차원 벡터 데이터 처리에 최적화
- 대규모 데이터셋에서도 안정적이고 빠른 검색 성능 보장
- 벡터를 효율적으로 인덱싱하여 고성능 검색과 메모리 효율성 제공

※ 인덱싱 방법으로는 IndexIVF-PQ 사용

- 리트리버 사용


# User 기반 추천 시스템(행렬 분해 모델)

## 가게 이름 추가

In [ ]:
# -*- coding: utf-8 -*-
# -*- coding: utf-8 -*-
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # BLAS 스레드 제한

import re
import pandas as pd
import numpy as np
from glob import glob
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# 1) 데이터 로드 및 컬럼 정리
df = pd.read_csv('/content/drive/MyDrive/V1_Recommend System/관광지 리뷰 크롤링/관광지_리뷰_크롤링_ALL.csv')

# 2) 컬럼명 통일 및 결측치 제거
#   - 리뷰가 없는 행은 제거하지만, 가게명은 그대로 보존합니다.
df = df.dropna(subset=['리뷰내용'])
df = df.rename(columns={
    '리뷰내용': 'review'
})
df['review'] = df['review'].astype(str)

# 2) 불용어 사전
STOPWORDS = set(['이','가','은','는','도','에','와','과','로','으로','의','를','을','하다'])

# 3) 형태소 분석기
okt = Okt()
def preprocess(text: str) -> str:
    text = re.sub(r'[^\w\s]', ' ', text)  # 특수문자 제거
    text = re.sub(r'\d+', ' ', text)      # 숫자 제거
    return re.sub(r'\s+', ' ', text).strip()

def tokenize(text: str) -> list[str]:
    pos = okt.pos(text, stem=True)
    return [
        word for word, tag in pos
        if tag in ['Noun','Adjective','Verb'] and word not in STOPWORDS
    ]

# 4) 키워드 추출 함수 (SBERT)
KW_MODEL = SentenceTransformer('jhgan/ko-sroberta-multitask', device='cuda')
def extract_keywords(doc: str, top_k: int = 5) -> list[str]:
    # 전처리 & 토크나이즈
    clean = preprocess(doc)
    toks  = tokenize(clean)
    if not toks:
        return []

    # TF-IDF 로 후보 만들기 (에러 방어)
    try:
        tfidf = TfidfVectorizer(ngram_range=(1,1)).fit([' '.join(toks)])
        candidates = tfidf.get_feature_names_out()
    except ValueError:
        return []

    if len(candidates) == 0:
        return []

    # 실제 뽑을 키워드 개수
    k = min(top_k, len(candidates))

    # 문장과 후보 임베딩
    emb_doc = KW_MODEL.encode(doc, convert_to_tensor=True)
    emb_kw  = KW_MODEL.encode(candidates, convert_to_tensor=True)

    # 코사인 유사도 상위 k개
    scores = util.pytorch_cos_sim(emb_doc, emb_kw)[0]
    topk_idx = scores.topk(k).indices.cpu().numpy()
    return [candidates[i] for i in topk_idx]

# 5) 감성분석 파이프라인
sentiment = pipeline(
    'sentiment-analysis',
    model='monologg/koelectra-base-v3-discriminator',
    tokenizer='monologg/koelectra-base-v3-discriminator',
    device= 0
)

# 6) DataFrame에 적용
df['cleaned']   = df['review'].map(preprocess)
df['tokens']    = df['cleaned'].map(tokenize)
df['keywords']  = df['review'].map(lambda x: extract_keywords(x, top_k=5))
df['sentiment'] = df['review'].map(lambda x: sentiment(x)[0]['label'])

# 7) 결과 확인
print(df[['가게이름','카테고리','전체평점','방문자리뷰','리뷰작성자','이런_점이_좋아요','방문시간','review','keywords','sentiment']].head())

# 8) CSV로 저장
df.to_csv('/content/drive/MyDrive/Sample/Store Name + 숙박_all_reviews_with_keywords_sentiment.csv', index=False)


In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Sample/Store Name + 숙박_all_reviews_with_keywords_sentiment.csv')

data

In [ ]:
import pandas as pd
from collections import Counter

# 리뷰 데이터 (예시)
# df에는 최소 다음 컬럼들이 있어야 합니다: '가게이름', 'keywords'
# 'keywords'는 각 리뷰에 대해 추출된 키워드 리스트 형태 (예: ['조용하다', '깨끗하다', '가깝다'])

# 1. 결측치 제거 및 확인
df_keywords = data.dropna(subset=['가게이름', 'keywords'])
df_keywords['keywords'] = df_keywords['keywords'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# 2. 장소별 키워드 누적
store_keywords = df_keywords.groupby('가게이름')['keywords'].sum()

# 3. 각 장소에서 상위 5개 키워드 추출
top_keywords_per_store = store_keywords.apply(lambda x: [kw for kw, _ in Counter(x).most_common(8)])

# 4. DataFrame으로 변환
top_keywords_df = top_keywords_per_store.reset_index()
top_keywords_df.columns = ['가게이름', '상위_키워드']
top_keywords_df.to_csv('/content/drive/MyDrive/Sample/25_06_13_Store Name + 숙박_all_reviews_with_keywords_sentiment.csv', index=False)

# 결과 확인
print(top_keywords_df.head(50))


## 리뷰 키워드 추출_관광지


In [ ]:
pip install sentence-transformers

In [ ]:
# -*- coding: utf-8 -*-
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # BLAS 스레드 제한

import re
import pandas as pd
import numpy as np
from glob import glob
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# 1) 데이터 로드 및 컬럼 정리
file_paths = glob('/content/drive/MyDrive/Sample_관광지/네이버 지도 방문자 리뷰 크롤러_관광지_*.xlsx')

df = pd.concat([
    pd.read_excel(
        fp,
        usecols=[
            '리뷰내용','가게이름','카테고리','전체평점',
            '방문자리뷰','리뷰작성자','이런_점이_좋아요','방문시간'
        ],
        engine='openpyxl'     # openpyxl 엔진 사용
    )
    for fp in file_paths
], ignore_index=True)

# 2) 컬럼명 통일 및 결측치 제거
#   - 리뷰가 없는 행은 제거하지만, 가게명은 그대로 보존합니다.
df = df.dropna(subset=['리뷰내용'])
df = df.rename(columns={
    '리뷰내용': 'review'
})
df['review'] = df['review'].astype(str)

# 2) 불용어 사전
STOPWORDS = set(['이','가','은','는','도','에','와','과','로','으로','의','를','을','하다'])

# 3) 형태소 분석기
okt = Okt()
def preprocess(text: str) -> str:
    text = re.sub(r'[^\w\s]', ' ', text)  # 특수문자 제거
    text = re.sub(r'\d+', ' ', text)      # 숫자 제거
    return re.sub(r'\s+', ' ', text).strip()

def tokenize(text: str) -> list[str]:
    pos = okt.pos(text, stem=True)
    return [
        word for word, tag in pos
        if tag in ['Noun','Adjective','Verb'] and word not in STOPWORDS
    ]

# 4) 키워드 추출 함수 (SBERT)
KW_MODEL = SentenceTransformer('jhgan/ko-sroberta-multitask')
def extract_keywords(doc: str, top_k: int = 5) -> list[str]:
    # 전처리 & 토크나이즈
    clean = preprocess(doc)
    toks  = tokenize(clean)
    if not toks:
        return []

    # TF-IDF 로 후보 만들기 (에러 방어)
    try:
        tfidf = TfidfVectorizer(ngram_range=(1,1)).fit([' '.join(toks)])
        candidates = tfidf.get_feature_names_out()
    except ValueError:
        return []

    if len(candidates) == 0:
        return []

    # 실제 뽑을 키워드 개수
    k = min(top_k, len(candidates))

    # 문장과 후보 임베딩
    emb_doc = KW_MODEL.encode(doc, convert_to_tensor=True)
    emb_kw  = KW_MODEL.encode(candidates, convert_to_tensor=True)

    # 코사인 유사도 상위 k개
    scores = util.pytorch_cos_sim(emb_doc, emb_kw)[0]
    topk_idx = scores.topk(k).indices.cpu().numpy()
    return [candidates[i] for i in topk_idx]

# 5) 감성분석 파이프라인
sentiment = pipeline(
    'sentiment-analysis',
    model='monologg/koelectra-base-v3-discriminator',
    tokenizer='monologg/koelectra-base-v3-discriminator',
    device=-1
)

# 6) DataFrame에 적용
df['cleaned']   = df['review'].map(preprocess)
df['tokens']    = df['cleaned'].map(tokenize)
df['keywords']  = df['review'].map(lambda x: extract_keywords(x, top_k=5))
df['sentiment'] = df['review'].map(lambda x: sentiment(x)[0]['label'])

# 7) 결과 확인
print(df[['가게이름','카테고리','전체평점','방문자리뷰','리뷰작성자','이런_점이_좋아요','방문시간','review','keywords','sentiment']].head())

# 8) CSV로 저장
df.to_csv('/content/drive/MyDrive/Sample_관광지/Store Name + 관광지_all_reviews_with_keywords_sentiment.csv', index=False)


In [ ]:
import pandas as pd
data_spot = pd.read_csv('/content/drive/MyDrive/Sample_관광지/Store Name + 관광지_all_reviews_with_keywords_sentiment.csv')

data_spot = df.dropna(subset=['keywords'])
data_spot.to_csv('/content/drive/MyDrive/Sample_관광지/Nan drop_Store Name + 관광지_all_reviews_with_keywords_sentiment.csv', index=False)



In [ ]:
df_temp = pd.read_csv('/content/drive/MyDrive/Sample/25_06_13_Store Name + 숙박_all_reviews_with_keywords_sentiment.csv')
df_temp

In [ ]:
# -*- coding: utf-8 -*-
# 06.13
import pandas as pd
from ast import literal_eval
from sentence_transformers import SentenceTransformer, util
from collections import Counter
import numpy as np
from glob import glob

# 1) 리뷰+키워드 파일 불러오기
paths = glob('/content/drive/MyDrive/Sample/Store Name + 숙박_all_reviews_with_keywords_sentiment.csv')
df_list = [pd.read_csv(p, dtype={'keywords': str}) for p in paths]
df = pd.concat(df_list, ignore_index=True)
# “['키워드1','키워드2']”처럼 문자열로 된 리스트를 진짜 리스트로 변환
df['keywords'] = df['keywords'].map(literal_eval)

# 2) 장소별 Top-N 키워드 집계 함수 (NaN·빈리스트 걸러짐)
def aggregate_keywords(df, store_col='가게이름', kw_col='keywords', top_n=5):
    # 1) kw_col이 list인 행만
    df2 = df[df[kw_col].map(lambda x: isinstance(x, list))]
    # 2) explode + NaN 걸러
    df2 = df2.explode(kw_col)
    df2 = df2[df2[kw_col].notna()]
    # 3) 장소별 리스트 집계
    grouped = df2.groupby(store_col)[kw_col].apply(list)
    # 4) Counter로 top_n 뽑아서 문장으로
    rows = []
    for store, kws in grouped.items():
        top = Counter(kws).most_common(top_n)
        # top 이 [(kw, cnt), ...] 이므로
        keywords = [kw for kw, _ in top]
        rows.append({'store_name': store,
                     'keyword_text': ' '.join(keywords)})
    return pd.DataFrame(rows)

# 3) SBERT 유사도 계산 함수
def compute_similarity(store_kw_df, model_name='jhgan/ko-sroberta-multitask'):
    model = SentenceTransformer(model_name)
    texts = store_kw_df['keyword_text'].tolist()
    emb = model.encode(texts, convert_to_tensor=True)
    sim_mat = util.pytorch_cos_sim(emb, emb).cpu().numpy()
    return sim_mat, store_kw_df['store_name'].tolist()

# 4) 실행
df_store_kw = aggregate_keywords(df, store_col='가게이름', kw_col='keywords', top_n=5)
sim_matrix, store_names = compute_similarity(df_store_kw)

# 5) 예시 출력: 첫 번째 가게와 가장 유사한 5곳
query_idx = 0
top5 = np.argsort(-sim_matrix[query_idx])[1:6]
for i in top5:
    print(f"{store_names[query_idx]} ↔ {store_names[i]} : {sim_matrix[query_idx][i]:.4f}")


In [ ]:
import pandas as pd

# (이전에 계산한 결과)
# df_store_kw: store_name, keyword_text
# sim_matrix: (N,N) numpy array
# store_names: store_name 리스트

# 1) DataFrame 으로 변환
sim_df = pd.DataFrame(
    data=sim_matrix,
    index=store_names,
    columns=store_names
)

# 2) 전체 보기
# Jupyter 환경이면 그냥 sim_df 를 마지막 줄에 두면 표시됩니다.
display(sim_df)

# 3) CSV 로 저장 (원하는 경로로 바꿔주세요)
sim_df.to_csv('/content/drive/MyDrive/Sample/store_keyword_similarity.csv', encoding='utf-8-sig')


In [ ]:
sim_df.iloc[-7:-1,-7:-1]

,화조원,환상숲곶자왈공원,휘닉스 아일랜드,휘닉스 제주 글라스 하우스,휴애리자연생활공원,흑백사진관 만나
화조원,1.000000,0.238382,0.399100,0.305697,0.274134,0.401051
환상숲곶자왈공원,0.238382,1.000000,0.500789,0.433856,0.441603,0.387127
휘닉스 아일랜드,0.399100,0.500789,1.000000,0.402024,0.389969,0.450644
휘닉스 제주 글라스 하우스,0.305697,0.433856,0.402024,1.000000,0.406498,0.441448
휴애리자연생활공원,0.274134,0.441603,0.389969,0.406498,1.000000,0.556805
흑백사진관 만나,0.401051,0.387127,0.450644,0.441448,0.556805,1.000000


In [ ]:
import pandas as pd

# sim_df: index/columns 모두 store_name 으로 된 (N × N) 유사도 행렬

def top_k_similar(sim_df: pd.DataFrame, k: int = 5) -> pd.DataFrame:
    """
    각 장소별로 자신을 제외하고 유사도 상위 k개 장소를 반환합니다.
    반환형은 store_name, similar_store, similarity 의 3컬럼 long-form DataFrame입니다.
    """
    records = []
    for store in sim_df.index:
        # 자신(store) 행에서 자신 컬럼은 제외
        row = sim_df.loc[store].drop(store)
        # 내림차순으로 정렬하고 상위 k개 선택
        topk = row.sort_values(ascending=False).head(k)
        for other_store, score in topk.items():
            records.append({
                'store_name':     store,
                'similar_store':  other_store,
                'similarity':     score
            })
    return pd.DataFrame.from_records(records)

# 사용 예
top5_df = top_k_similar(sim_df, k=5)
print(top5_df.head(50))
# 만약 엑셀/CSV로 저장하고 싶다면:
top5_df.to_csv('store_top5_similar.csv', index=False, encoding='utf-8-sig')


In [ ]:
import pandas as pd


meta_df = pd.read_csv('/content/drive/MyDrive/Meta Data/jeju_tour_spot.csv')
reviews_df = pd.read_csv('/content/drive/MyDrive/V1_Recommend System/관광지 리뷰 크롤링/관광지_리뷰_크롤링_ALL.csv')

print(list(meta_df.columns))
print(list(reviews_df.columns))


['contents_id', 'contents_label', 'title', 'address', 'road_address', 'tag', 'introduction', 'latitude', 'longitude', '평일오픈시간', '평일클로즈시간', '주말오픈시간', '주말클로즈시간']
['가게이름', '카테고리', '전체평점', '방문자리뷰', '리뷰작성자', 'review', '이런_점이_좋아요', '방문시간', 'cleaned', 'tokens', 'keywords', 'sentiment']


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/V1_Recommend System/관광지 리뷰 크롤링/관광지_리뷰_크롤링_ALL.csv')

# 1) 방문시간 문자열화
df['방문시간'] = df['방문시간'].astype(str)

# 2) 연도 유무 판단하여 기본 연도('24') 붙이기
has_year = df['방문시간'].str.contains(r'^\d{2}\.\d{1,2}\.\d{1,2}')
df.loc[~has_year, '방문시간'] = '24.' + df.loc[~has_year, '방문시간']

# 3) 날짜 부분(yy.mm.dd)만 추출
df['date_str'] = df['방문시간'].str.extract(r'(\d{2}\.\d{1,2}\.\d{1,2})')[0]

# 4) datetime 변환
df['방문일시'] = pd.to_datetime(df['date_str'], format='%y.%m.%d', errors='coerce')

# 5) 분기 컬럼 추가
df['quarter'] = df['방문일시'].dt.quarter

# 6) 중간필드 정리 (원하면)
df.drop(columns=['date_str'], inplace=True)

# 7) 결과를 CSV로 저장
df.to_csv('/content/drive/MyDrive/Sample_관광지/Store Name + 관광지_all_reviews_with_quarter.csv', index=False, encoding='utf-8-sig')


In [ ]:
df.columns

Index(['가게이름', '카테고리', '전체평점', '방문자리뷰', '리뷰작성자', 'review', '이런_점이_좋아요', '방문시간',
       'cleaned', 'tokens', 'keywords', 'sentiment', '방문일시', 'quarter'],
      dtype='object')

## Co-Review 기반 추천 시스템

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, util
from math import radians, sin, cos, sqrt, atan2
import lightgbm as lgb
import torch

# 1) 데이터 불러오기 -------------------------------------------------------

meta_df = pd.read_csv('/content/drive/MyDrive/Meta Data/jeju_tour_spot.csv',
    usecols=[
      'contents_id','contents_label','title','address','road_address','tag',
      'introduction','latitude','longitude',
      '평일오픈시간','평일클로즈시간','주말오픈시간','주말클로즈시간'
    ]
)
reviews_df = pd.read_csv('/content/drive/MyDrive/Sample_관광지/Store Name + 관광지_all_reviews_with_quarter.csv',
    usecols=[
      '가게이름','카테고리','전체평점','방문자리뷰',
      '리뷰작성자','review','이런_점이_좋아요','방문시간',
      'cleaned','tokens','keywords','sentiment','quarter'
    ]
)

meta_map = meta_df[['contents_id','title']].rename(columns={'title':'가게이름'})
reviews_df = reviews_df.merge(meta_map, on='가게이름', how='left')

# 2) 장소별 키워드 집계 -------------------------------------------------------
def aggregate_top_keywords(df, place_col='contents_id', kw_col='keywords', top_n=5):
    # 리뷰별 keywords 리스트 explode
    exploded = df[[place_col, kw_col]].explode(kw_col).dropna(subset=[kw_col])
    counts = (
      exploded
      .groupby([place_col, kw_col])
      .size()
      .reset_index(name='cnt')
    )
    # 장소별 상위 top_n 키워드만 추출
    topk = (
      counts
      .sort_values([place_col,'cnt'], ascending=[True,False])
      .groupby(place_col)
      .head(top_n)
      .groupby(place_col)[kw_col]
      .apply(list)
      .reset_index()
      .rename(columns={kw_col:'top_keywords'})
    )
    # 리스트를 공백으로 합쳐서 한 문장으로
    topk['keyword_text'] = topk['top_keywords'].map(lambda kws: " ".join(kws))
    return topk[[place_col,'keyword_text']]

store_kw = aggregate_top_keywords(reviews_df, place_col='contents_id', kw_col='keywords', top_n=5)
meta_df = meta_df.merge(store_kw, on='contents_id', how='left').fillna({'keyword_text':''})


# 3) SBERT 임베딩 & 유사도 ----------------------------------------------------
sbert = SentenceTransformer('jhgan/ko-sroberta-multitask')
# 장소별 키워드 문장 임베딩 (tensor list)
meta_df['kw_emb'] = list(
    sbert.encode(
      meta_df['keyword_text'].tolist(),
      convert_to_tensor=True
    )
)
# 1) 모든 임베딩을 (N, D) 형태로 쌓기
embs = torch.stack(meta_df['kw_emb'].tolist(), dim=0)

# 2) N×N 코사인 유사도 행렬 계산
sim_matrix = util.pytorch_cos_sim(embs, embs).cpu().numpy()


# 4) Haversine 거리 계산 -----------------------------------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # 지구 반경 (km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1))*cos(radians(lat2))*sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1-a))

coords = meta_df[['latitude','longitude']].to_numpy()
n = len(coords)
dist_matrix = np.zeros((n,n), dtype=float)
for i in range(n):
    for j in range(i+1, n):
        d = haversine(*coords[i], *coords[j])
        dist_matrix[i,j] = dist_matrix[j,i] = d


# 5) Co-Review Count 생성 ----------------------------------------------------
# 같은 (year,quarter,user_id)에 리뷰를 남긴 장소 쌍 수를 센다.
pair_counter = Counter()
grp = (
    reviews_df
    .dropna(subset=['contents_id','review'])
    .groupby(['quarter','리뷰작성자'])
    ['contents_id']
    .apply(lambda lst: sorted(set(lst)))
)
for places in tqdm(grp, desc="building co-review"):
    for i, j in combinations(places, 2):
        pair_counter[(i,j)] += 1

co_pairs = [
    {'place1': i, 'place2': j, 'co_count': cnt}
    for (i,j), cnt in pair_counter.items()
]
co_df = pd.DataFrame(co_pairs)


# 6) 피처 테이블 생성 --------------------------------------------------------
# place_id → meta_df 인덱스 매핑
place2idx = {pid: idx for idx, pid in enumerate(meta_df['contents_id'])}

def make_feature_table(co_df, meta_df):
    rows = []
    for _, r in tqdm(co_df.iterrows(), total=len(co_df), desc="feature table"):
        i, j = r['place1'], r['place2']
        if i not in place2idx or j not in place2idx:
            continue
        idx_i, idx_j = place2idx[i], place2idx[j]
        rows.append({
            'place1': i,
            'place2': j,
            'co_count': r['co_count'],
            'dist_km':    dist_matrix[idx_i, idx_j],
            'sbert_sim':  sim_matrix[idx_i, idx_j],
        })
    return pd.DataFrame(rows)

feat_df = make_feature_table(co_df, meta_df)


# 7) LightGBM 학습 파이프라인 예시 ------------------------------------------
# 회귀(regression) 혹은 분류(classification) 태스크로 바꾸셔도 됩니다.
X = feat_df.drop(columns=['place1','place2','co_count'])
y = feat_df['co_count']

lgb_train = lgb.Dataset(X, y)
params = {
    'objective':'regression',
    'metric':'rmse',
    'learning_rate':0.1,
}
gbm = lgb.train(params, lgb_train, num_boost_round=200)


# 8) 결과 저장 ---------------------------------------------------------------
# 메타: 키워드 문장만 남기고 embedding은 삭제
meta_df.drop(columns=['kw_emb'], inplace=True)
meta_df.to_csv('meta_with_kw.csv', index=False)
feat_df.to_csv('item_cf_features.csv', index=False)

print("파이프라인 완료")

building co-review:   0%|          | 0/7027 [00:00<?, ?it/s]

feature table:   0%|          | 0/677 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000279 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 292
[LightGBM] [Info] Number of data points in the train set: 677, number of used features: 2
[LightGBM] [Info] Start training from score 2.587888
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

## 컬럼 확인용

In [ ]:
df1 = pd.read_csv('/content/meta_with_kw.csv')

df2 = pd.read_csv('/content/item_cf_features.csv')

df2

,place1,place2,co_count,dist_km,sbert_sim
0,CONT_000000000500293,CONT_000000000500307,1,6.205368,0.650023
1,CONT_000000000500293,CONT_000000000500597,1,15.049237,0.429435
2,CONT_000000000500307,CONT_000000000500597,1,21.251134,0.491735
3,CNTS_000000000021240,CONT_000000000500356,5,14.754173,0.645431
4,CNTS_200000000008155,CONT_000000000500002,3,43.052945,0.748061
...,...,...,...,...,...
672,CONT_000000000500232,CONT_000000000500406,1,5.603557,0.765089
673,CONT_000000000500374,CONT_000000000500406,1,2.271028,0.819641
674,CNTS_000000000021240,CONT_000000000500488,1,22.585027,0.773445
675,CONT_000000000500232,CONT_000000000500488,1,20.353625,0.777279


높은 co_count + 작은 dist_km + 높은 sbert_sim 조합은
“여행 동선상 가깝고, 테마(주요 키워드)도 비슷해서 실제 많이 함께 방문된 장소 쌍”을 의미.

반면에 co_count는 낮지만 sbert_sim이 매우 높다면 “주제·분위기는 비슷하나 지리적으로 떨어져 있는, ‘테마’ 추천용 후보”,

co_count는 높은데 sbert_sim이 낮으면 “사용자들이 함께 가긴 했지만 리뷰 키워드는 다르게 기술한, 묶어서 추천할 땐 주의해야 할 쌍”.

##컬럼 확인용





In [ ]:
meta_df2 = pd.read_csv('/content/drive/MyDrive/Meta Data/jeju_tour_spot.csv')

review_df2 =  pd.read_csv('/content/drive/MyDrive/Sample/25_05_10_Store Name + 숙박_all_reviews_with_keywords_sentiment.csv')
review_df2

## 키워드 + Text 관광지 추천 시스템

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
meta_df = pd.read_csv('/content/drive/MyDrive/Meta Data/jeju_tour_spot.csv',
    usecols=[
        'contents_id','title',
        'tag','introduction'
    ]
)

#    키워드 텍스트 불러오기
kw_df = pd.read_csv('/content/drive/MyDrive/Sample/25_05_10_Store Name + 숙박_all_reviews_with_keywords_sentiment.csv',
    usecols=['가게이름','상위_키워드']
)

kw_df = kw_df.rename(columns={
    '가게이름':     'title',
    '상위_키워드': 'keyword_text'
})
# 병합 (LEFT JOIN)
df = meta_df.merge(kw_df, on='title', how='left')

# 누락된 키워드는 빈 문자열로
df['keyword_text'] = df['keyword_text'].fillna('')

# tag와 keyword_text 합쳐서 추천용 텍스트 생성
df['combined_text'] = (
    df['tag'].fillna('') + ' ' + df['keyword_text']
).str.strip()


# 1) SBERT 모델 로드
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 2) 사용자가 입력한 키워드 리스트 예시
user_kw = ['산책', '해안도로','드라이브']
user_sent = " ".join(user_kw)
emb_user = model.encode(user_sent, convert_to_tensor=True)


# 3) 장소별 combined_text 임베딩 (한 번만 수행)
emb_corpus = model.encode(
    df['combined_text'].tolist(),
    convert_to_tensor=True
)

cos_scores = util.pytorch_cos_sim(emb_user, emb_corpus)[0]  # tensor of length N
top_k = 10
top_results = torch.topk(cos_scores, k=top_k)

# 6) 결과 출력
top_indices = top_results.indices.cpu().numpy()
top_scores  = top_results.values.cpu().numpy()
recommendations = df.iloc[top_indices].copy()
recommendations['score'] = top_scores

# 최종 추천 리스트 보기
print(recommendations[['contents_id','title','score']])

               contents_id     title     score
114   CNTS_000000000022390   병악현무암지대  0.795793
1109  CNTS_000000000021473     구좌해안로  0.733295
234   CNTS_000000000021407      옹포포구  0.731514
396   CNTS_000000000021460      돔베낭길  0.720637
1073  CNTS_200000000007919  행원육상양식단지  0.697998
482   CNTS_200000000008054  고내리 다락쉼터  0.696653
1056  CONT_000000000500526       절부암  0.682185
1103  CNTS_000000000019012    형제해안도로  0.674627
342   CNTS_000000000019534    닭머르해안길  0.671746
364   CNTS_200000000007346      설쿰바당  0.670689


In [ ]:
import pandas as pd

# 파일 경로
path_visit = "/content/drive/MyDrive/전처리 데이터/visit_jeju.csv"
path_info  = "/content/drive/MyDrive/V3_관광지 카테고리/V3_관광지Data_권역추가.csv"

# 1) 데이터 로드
df_visit = pd.read_csv(path_visit, dtype=str)
df_info  = pd.read_csv(path_info, dtype=str)

# 2) 컬럼명 표준화
df_visit.rename(columns={'이름': 'VISIT_AREA_NM'}, inplace=True)

# 3) 병합: visit 데이터에 테마 컬럼 추가
df_merged = pd.merge(
    df_visit[['VISIT_AREA_NM', '테마', '주소']],
    df_info[['VISIT_AREA_NM', '소분류', '세분류', 'TRAVEL_ID', 'DGSTFN']],
    on='VISIT_AREA_NM',
    how='left'
)

# 4) 결과 확인
print(df_merged.head())